In [9]:
import minimalmodbus
DEVICE_PORT = 'COM5'
SLAVE_ADDRESS = 1
instrument = None  # Define instrument outside the try block
print(f"--- Starting Communication Test on {DEVICE_PORT} ---")
try:
    # 1. Initialize the instrument object
    instrument = minimalmodbus.Instrument(DEVICE_PORT, SLAVE_ADDRESS)
    # 2. Set all communication parameters
    instrument.serial.baudrate = 9600
    instrument.serial.bytesize = 8
    instrument.serial.parity = minimalmodbus.serial.PARITY_NONE
    instrument.serial.stopbits = 2
    instrument.mode = minimalmodbus.MODE_RTU
    
    # Set a short timeout to make the test faster
    instrument.serial.timeout = 0.5 
    
    # 3. Attempt to read one register (Voltage, register 0)
    print("Attempting to read from the sensor...")

    # This is the single command we are testing.
    # It must use functioncode=4 for the PZEM-017.
    voltage_value = instrument.read_register(
        registeraddress=0, 
        number_of_decimals=2, 
        functioncode=4
    )
    print("\n✅ ✅ ✅ SUCCESS! ✅ ✅ ✅")
    print("Communication with the PZEM-017 is working.")
    print(f"Received a voltage reading of: {voltage_value} V")

except Exception as e:
    print("\n❌ ❌ ❌ COMMUNICATION FAILED! ❌ ❌ ❌")
    print("The script did not receive a reply from the sensor.")
    print("This confirms the problem is with the physical connection or setup.")
    print(f"Error details: {e}\n")
    print("Next steps: Re-check power to the PZEM and swap the A/B data wires.")

finally:
    # 6. Ensure the port is always closed
    if instrument:
        instrument.serial.close()
    print("\n--- Test Finished ---")

--- Starting Communication Test on COM5 ---
Attempting to read from the sensor...

✅ ✅ ✅ SUCCESS! ✅ ✅ ✅
Communication with the PZEM-017 is working.
Received a voltage reading of: 24.01 V

--- Test Finished ---


In [10]:
import minimalmodbus
import time
import datetime
import csv
import os

# --- CONFIGURATION ---
DEVICE_PORT = 'COM5'
SLAVE_ADDRESS = 1
LOG_FILENAME = "energy_log.csv"

instrument = None 

try:
    # --- Create CSV and write header if file does not exist ---
    new_file = not os.path.exists(LOG_FILENAME)
    logfile = open(LOG_FILENAME, "a", newline="")
    csv_writer = csv.writer(logfile)

    if new_file:
        csv_writer.writerow([
            "Timestamp",
            "Voltage (V)",
            "Current (A)",
            "Power (W)",
            "Delta Time (s)",
            "Cumulative Energy (Wh)"
        ])

    # --- Initialize Modbus ---
    instrument = minimalmodbus.Instrument(DEVICE_PORT, SLAVE_ADDRESS)
    instrument.serial.baudrate = 9600
    instrument.serial.bytesize = 8
    instrument.serial.parity = minimalmodbus.serial.PARITY_NONE
    instrument.serial.stopbits = 2
    instrument.mode = minimalmodbus.MODE_RTU
    instrument.close_port_after_each_call = True
    
    # --- Time & Energy Tracking ---
    start_time = time.time()
    start_timestamp = datetime.datetime.now()
    last_energy_time = start_time
    total_energy_Wh = 0.0

    print(f"Successfully connected to PZEM-017 on port {DEVICE_PORT}")
    print(f"Logging started at: {start_timestamp.strftime('%Y-%m-%d %H:%M:%S')}")
    print("\nLogging to CSV file:", LOG_FILENAME)
    print("Press CTRL+C to stop.\n")

    while True:
        try:
            data = instrument.read_registers(0, 6, 4)

            voltage = data[0] / 100.0
            current = data[1] / 100.0
            power = ((data[3] << 16) | data[2]) / 10.0

            now = time.time()
            dt = now - last_energy_time
            if dt > 0:
                total_energy_Wh += power * dt / 3600.0
                last_energy_time = now

            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            # Write row to CSV
            csv_writer.writerow([timestamp, voltage, current, power, round(dt, 3), total_energy_Wh])
            logfile.flush()   # ensures real-time saving

            print(f"{timestamp} | {voltage:.2f}V | {current*1000:.1f} mA | {power:.2f}W | "
                  f"Energy: {total_energy_Wh:.4f} Wh")

        except Exception as e:
            print(f"WARNING: A read error occurred. Error: {e}")
        
        time.sleep(2)

except KeyboardInterrupt:
    end_time = time.time()
    end_timestamp = datetime.datetime.now()
    duration_sec = end_time - start_time

    hours = int(duration_sec // 3600)
    minutes = int((duration_sec % 3600) // 60)
    seconds = int(duration_sec % 60)

    print("\n--- Logging stopped by user ---")
    print(f"Start time : {start_timestamp}")
    print(f"End time   : {end_timestamp}")
    print(f"Total time : {hours:02d}:{minutes:02d}:{seconds:02d}")
    print(f"Total Energy: {total_energy_Wh:.4f} Wh ({total_energy_Wh/1000:.6f} kWh)")
    print(f"Data saved to: {LOG_FILENAME}")

except Exception as e:
    print(f"--- CRITICAL ERROR --- \n{e}")

finally:
    try:
        logfile.close()
    except:
        pass

    try:
        if instrument and instrument.serial and instrument.serial.is_open:
            instrument.serial.close()
    except:
        pass


Successfully connected to PZEM-017 on port COM5
Logging started at: 2025-11-29 23:18:27

Logging to CSV file: energy_log.csv
Press CTRL+C to stop.

2025-11-29 23:18:28 | 24.01V | 1620.0 mA | 38.80W | Energy: 0.0011 Wh
2025-11-29 23:18:30 | 24.02V | 1620.0 mA | 38.90W | Energy: 0.0240 Wh
2025-11-29 23:18:32 | 24.01V | 1620.0 mA | 38.80W | Energy: 0.0468 Wh
2025-11-29 23:18:34 | 24.01V | 1650.0 mA | 39.60W | Energy: 0.0701 Wh
2025-11-29 23:18:36 | 24.02V | 1620.0 mA | 38.90W | Energy: 0.0932 Wh
2025-11-29 23:18:38 | 24.01V | 1620.0 mA | 38.80W | Energy: 0.1160 Wh
2025-11-29 23:18:40 | 24.01V | 1620.0 mA | 38.80W | Energy: 0.1389 Wh
2025-11-29 23:18:42 | 24.01V | 1650.0 mA | 39.60W | Energy: 0.1623 Wh
2025-11-29 23:18:44 | 24.02V | 1650.0 mA | 39.60W | Energy: 0.1856 Wh
2025-11-29 23:18:47 | 24.02V | 1140.0 mA | 27.30W | Energy: 0.2017 Wh
2025-11-29 23:18:49 | 24.01V | 1500.0 mA | 36.00W | Energy: 0.2230 Wh
2025-11-29 23:18:51 | 24.01V | 1620.0 mA | 38.80W | Energy: 0.2458 Wh
2025-11-29 2